In [50]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from copy import copy, deepcopy
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler



In [51]:
pd.get_option("display.max_columns")
df2 = pd.read_csv("cup98lrn.txt", sep=',', error_bad_lines = False, low_memory = False, skip_blank_lines = True)
df1 = pd.read_csv("cup98lrn.txt", sep=',', error_bad_lines = False, low_memory = False, skip_blank_lines = True)

In [ ]:
def run_regression(data):
    # train test split
    train, test = train_test_split(data, test_size=0.3)
    # get targets out
    train_B = train["TARGET_B"]
    train_D = train["TARGET_D"]
    test_B = test["TARGET_B"]
    test_D = test["TARGET_D"]
    train.drop(columns = ["TARGET_D","TARGET_B"], inplace = True)
    
    # we need to resample the train data to balance it out
    sampler = RandomOverSampler(random_state=50)
    x_res, y_res = over_sampler.fit_resample(train, train_B)
    print(f"Training target statistics: {Counter(y_res)}")
    print(f"Testing target statistics: {Counter(y_test)}")
    
    # train the model
    clf = LogisticRegression(max_iter = 100, solver = "liblinear", verbose = 1)
    clf = clf.fit(x_res, y_res)
    
    # test on the test set
    y_pred = clf.predict(test)
    
    return get_acc(y_pred, test_B, test_D, 0.68)

In [ ]:
def get_acc(y_pred, y_actual, y_donate, mail_cost):
    df = pd.concat([y_pred, y_actual, y_donate], axis = 1)
    df.columns = ["y_pred", "y_actual", "y_donate"]
    
    # get false positive rate
    fp_rate = df[(df['y_pred'] == 1) & (df['y_actual'] == 0)].shape[0]
    # get false negative rate
    fn_rate = df[(df['y_pred'] == 0) & (df['y_actual'] == 1)].shape[0]
    # get total profit 
    profit = df[(df['y_pred'] == 1) & (df['y_actual'] == 1)]["y_donate"].sum() - df[(df['y_pred'] == 1)].shape[0]*mail_cost
    
    return fp_rate, fn_rate, profit

In [4]:
'''
Data preprocessing tasks include the following(https://kdd.org/cupfiles/KDDCupData/1998/cup98doc.txt)

1. Noisy Data
Some of the fields in the analysis file may contain data entry and/or
formatting errors. You are expected to clean these fields (without
excluding the records.)
Rui: I think dataframe already takes care of this part. Dont think there is any data record as formatting issue

2. Records and Fields with Missing and Sparse Data
    2.1:Drop the attribute when it missing more than 99.5% 
    2.2:replace the missing record with the (mean,mode,avg, most common type, etc) 
            value of the avg. value of attribute
3.Fields Containing Constants
4.Time Frame and Date Fields     
'''

def trim_data(df):
    
    #I think 1 is already done by dataframe
    
    
    
    #there are a lot whitesapces in features. I replace whitespaces with nan first
    # for the convenience of data processing
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    
    num_cols = len(df.columns)
    num_rows = len(df)
    ####Fields Containing Constants################
    df.dropna(axis = 0, thresh= 2, inplace=True)
    
    ####dealing with missing values#################
        
    #1. drop the attribute if missing values >= 99.5%
    #calculating the dropping_treshold 
    perc = 99.5
    min_count =  int(((100-perc)/100)*num_rows+ 1)
    df.dropna(axis = 1, thresh=min_count, inplace=True)
    
    #2. if features contains NAN < 99.5% we need to replace NAN with the most frequent value
    #this line does replace differnet attribute types(Number, char, boolean, etc)  with the most frequent
    # value
    df.fillna(df.mode().iloc[0], inplace=True)
    
    
    ####dealing with sparse values#################
    
    
    ####Time Frame and Date Fields#########
    
    
    